In [ ]:
#!pip install pynput

In [1]:
import cv2
import mediapipe as mp
import pyautogui
from pynput.mouse import Button, Controller
import random

In [2]:
mouse = Controller()
screen_width, screen_height = pyautogui.size()

mpHands = mp.solutions.hands
hands = mpHands.Hands(
        static_image_mode=False,
        model_complexity=1,
        min_detection_confidence=0.7,  
        min_tracking_confidence=0.7,
        max_num_hands = 1
)

In [3]:
import numpy as np


def get_angle(a,b,c):
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(np.degrees(radians))
    return angle

def get_distance(landmark_list):
    if len(landmark_list) < 2:
        return
    (x1,y1), (x2,y2) = landmark_list[0], landmark_list[1]
    L = np.hypot(x2 - x1, y2 - y1)
    return np.interp(L, [0, 1], [0, 1000])

In [4]:
def find_finger_tip(processed):
    
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
    
    return None 

In [5]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_width)
        y = int(index_finger_tip.y * screen_height)
        pyautogui.moveTo(x,y)

In [6]:
def is_left_click(landmarks_list, thumb_index_dist):
    return (get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) < 50 and 
            get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) > 90 and
            thumb_index_dist > 50)

def is_right_click(landmarks_list, thumb_index_dist):
    return (get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) < 50 and 
            get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) > 90 and
            thumb_index_dist > 50)

def is_double_click(landmarks_list, thumb_index_dist):
    return (get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8])<50 and 
            get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12])<50 and
            thumb_index_dist > 50)

def is_screenshot(landmarks_list, thumb_index_dist):
    return (get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8])<50 and 
            get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12])<50 and
            thumb_index_dist < 50)

In [7]:
def detect_gestures(frame, landmarks_list, processed):
    if len(landmarks_list)>=21:
        index_finger_tip = find_finger_tip(processed)
        thumb_index_dist = get_distance([landmarks_list[4], landmarks_list[5]])
        
        if thumb_index_dist < 50 and get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8])>90:
            move_mouse(index_finger_tip)
            
        #left Click
        elif is_left_click(landmarks_list, thumb_index_dist):
            mouse.press(Button.left)
            mouse.release(Button.left)
            cv2.putText(frame, "Left Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0), 2)
                        
        #right click
        elif is_right_click(landmarks_list, thumb_index_dist):
            mouse.press(Button.right)
            mouse.release(Button.right)
            cv2.putText(frame, "Right Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255), 2)
            
        #double click
        elif is_double_click(landmarks_list, thumb_index_dist):
            pyautogui.doubleClick()
            cv2.putText(frame, "Double Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0), 2)
            
        #screenshot
        elif is_screenshot(landmarks_list, thumb_index_dist):
            im1 = pyautogui.screenshot()
            label = random.randint(1, 1000)
            im1.save(f'my_screenshot_{label}.png')
            cv2.putText(frame, "Screenshot Taken", (50, 50), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0), 2)
            

In [1]:
def main():
    cap = cv2.VideoCapture(0)
    # to draw hands line and points
    draw = mp.solutions.drawing_utils
    
    try:
        while cap.isOpened():
            ret , frame = cap.read()
            
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            # mediapipe requires frame to be passed in rgb format for processing
            # But OpenCV bydefault captures frame in bgr format so we need to change it to rgb format
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            processed = hands.process(frameRGB)
            
            landmarks_list = []
            
            if processed.multi_hand_landmarks:
                hand_landmarks = processed.multi_hand_landmarks[0]
                draw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)
                
                for lm in hand_landmarks.landmark:
                    landmarks_list.append((lm.x, lm.y))
            
            detect_gestures(frame, landmarks_list, processed)
                    
                
            
            cv2.imshow('Frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    finally:
        cap.release()
        cv2.destroyAllWindows()
        
if __name__ == '__main__':
    main()
        

NameError: name 'cv2' is not defined